In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd 'drive/My Drive/github-bug-classification'
%pwd

/content/drive/My Drive/github-bug-classification


'/content/drive/My Drive/github-bug-classification'

In [ ]:
%cd 'data'
!unzip 'archive.zip'
%cd ..

/content/drive/My Drive/github-bug-classification/data
Archive:  archive.zip
replace embold_test.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: embold_test.json        
replace embold_train.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: embold_train.json       
  inflating: embold_train_extra.json  
replace sample submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sample submission.csv   
/content/drive/My Drive/github-bug-classification


In [ ]:
!pip install transformers

     |████████████████████████████████| 1.8MB 8.4MB/s 
     |████████████████████████████████| 3.2MB 33.9MB/s 
     |████████████████████████████████| 890kB 57.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=0c42cc858523a9c4a87212546c12d2f9865e1731ee323767069bb0ce822e8b30
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import pandas as pd
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

In [ ]:
import torch

# Data tokenization

In [ ]:
train = pd.read_json('data/embold_train.json')
train_extra = pd.read_json('data/embold_train_extra.json')
train = pd.concat([train, train_extra], ignore_index=True)
test = pd.read_json('data/embold_test.json')
# 0: bug, 1: feature, 2: question

In [ ]:
train['text'] = train.title + ' ' + train.body
test['text'] = test.title + ' ' + test.body

train.head()

,title,body,label,text
0,y-zoom piano roll,a y-zoom on the piano roll would be useful.,1,y-zoom piano roll a y-zoom on the piano roll w...
1,buggy behavior in selection,! screenshot from 2016-02-23 21 27 40 https:/...,0,buggy behavior in selection ! screenshot from ...
2,auto update feature,"hi,\r \r great job so far, @saenzramiro ! : \r...",1,"auto update feature hi,\r \r great job so far,..."
3,filter out noisy endpoints in logs,i think we should stop logging requests to:\r ...,1,filter out noisy endpoints in logs i think we ...
4,enable pid on / pid off alarm actions for ardu...,expected behavior\r alarm actions pid on and p...,0,enable pid on / pid off alarm actions for ardu...


In [ ]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(train, test_size=.2)

In [ ]:
train_texts = train[:2000].text.to_list()#to_numpy()
val_texts = val[:2000].text.to_list()#to_numpy()
test_texts = val[-2000:].text.to_list()
#test_texts = test[:2000].text.to_list()#to_numpy()

In [ ]:
train_labels = train[:2000].label.to_list()
val_labels = val[:2000].label.to_list()
test_labels = val[-2000:].label.to_list()
#test_labels = test[:2000].label.to_list()

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Datasets

In [ ]:
import torch

class GitDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = GitDataset(train_encodings, train_labels)
val_dataset = GitDataset(val_encodings, val_labels)
test_dataset = GitDataset(test_encodings, test_labels)

# Fine-tuning

In [44]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    #precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        #'f1': f1,
        #'precision': precision,
        #'recall': recall
    }


In [ ]:
#model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

In [45]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=2,  # batch size per device during training
    per_device_eval_batch_size=4,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset
    compute_metrics = compute_metrics
)

In [ ]:
model.to('cuda')

In [42]:
trainer.train()

Step,Training Loss
10,1.095500
20,1.053300
30,1.010600
40,1.054500
50,0.904200
60,0.971700
70,1.004100
80,0.821100
90,1.128300
100,0.973600


TrainOutput(global_step=3000, training_loss=0.7293410239089281, metrics={'train_runtime': 869.6338, 'train_samples_per_second': 3.45, 'total_flos': 2018019170304000, 'epoch': 3.0})

In [46]:
trainer.evaluate()

{'eval_accuracy': 0.803,
 'eval_loss': 1.1148594617843628,
 'eval_runtime': 79.7345,
 'eval_samples_per_second': 25.083}